Specialization:


It appears that annotations in ``` get42 ``` are optional, in a sence jit produces the same result either with or w/o
annotations. All the work does the unfold function.

**use of range fun:**
```
struct Keys{
    array : [u8 * 9],
}


fn @get42(keys : Keys, values: &[u8], key: u8) -> u8 {
    
    let mut res = 0;

    range(0,9, |i| {
            
            if keys.array(i) == key {// == key
                    res = i;

    }
            });

    values(res)
    

}
```

**LLVM with opt = 3:**

```
 ; ModuleID = 'jit'
source_filename = "jit"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@_8009 = internal global [2 x i8] c"\0A\00"

define void @dummy([0 x i8]* nocapture readonly %vals_7949, i8 %key_7950) local_unnamed_addr {

dummy_start:
  %0 = getelementptr inbounds [0 x i8], [0 x i8]* %vals_7949, i64 0, i64 6
  %1 = load i8, i8* %0, align 1
  tail call void @anydsl_print_char(i8 %1)
  tail call void @anydsl_print_string([0 x i8]* bitcast ([2 x i8]* @_8009 to [0 x i8]*))
  ret void
}

declare void @anydsl_print_char(i8) local_unnamed_addr

declare void @anydsl_print_string([0 x i8]*) local_unnamed_addr 


```

**opt = 0;**

```
; ModuleID = 'jit'
source_filename = "jit"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@_8349 = internal global [2 x i8] c"\0A\00"
@_8314 = internal global [9 x i8] c"12345678\00"

define void @dummy([0 x i8]* %vals_8297) {
dummy_start:
  %res_8325 = alloca i32
  br label %dummy

dummy:                                            ; preds = %dummy_start
  store i32 0, i32* %res_8325, align 4
  br label %unroll_step

unroll_step:                                      ; preds = %_cont, %dummy
  %lower = phi i32 [ %7, %_cont ], [ 0, %dummy ]
  %0 = icmp slt i32 %lower, 9
  br i1 %0, label %expr_true, label %expr_false

expr_false:                                       ; preds = %unroll_step
  %1 = load i32, i32* %res_8325, align 4
  %2 = getelementptr inbounds [0 x i8], [0 x i8]* %vals_8297, i64 0, i32 %1
  %3 = load i8, i8* %2, align 1
  call void @anydsl_print_char(i8 %3)
  br label %anydsl_print_char_cont

anydsl_print_char_cont:                           ; preds = %expr_false
  call void @anydsl_print_string([0 x i8]* bitcast ([2 x i8]* @_8349 to [0 x i8]*))
  br label %return

return:                                           ; preds = %anydsl_print_char_cont
  ret void

expr_true:                                        ; preds = %unroll_step
  %4 = getelementptr inbounds [9 x i8], [9 x i8]* @_8314, i64 0, i32 %lower
  %5 = load i8, i8* %4
  %6 = icmp eq i8 %5, 55
  br i1 %6, label %expr_true2, label %expr_false1

expr_false1:                                      ; preds = %expr_true
  br label %_cont

expr_true2:                                       ; preds = %expr_true
  store i32 %lower, i32* %res_8325, align 4
  br label %_cont

_cont:                                            ; preds = %expr_true2, %expr_false1
  %7 = add nsw i32 1, %lower
  br label %unroll_step
}

declare void @anydsl_print_char(i8)

declare void @anydsl_print_string([0 x i8]*)
```

**unfold llvm opt = 3:**

```
; ModuleID = 'jit'
source_filename = "jit"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@_7246 = internal global [2 x i8] c"\0A\00"

define void @dummy([0 x i8]* nocapture readonly %vals_7224) local_unnamed_addr {
dummy_start:
  %0 = getelementptr inbounds [0 x i8], [0 x i8]* %vals_7224, i64 0, i64 6
  %1 = load i8, i8* %0, align 1
  tail call void @anydsl_print_char(i8 %1)
  tail call void @anydsl_print_string([0 x i8]* bitcast ([2 x i8]* @_7246 to [0 x i8]*))
  ret void
}

declare void @anydsl_print_char(i8) local_unnamed_addr

declare void @anydsl_print_string([0 x i8]*) local_unnamed_addr

```

**unfold llvm opt = 0:**

```
; ModuleID = 'jit'
source_filename = "jit"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@_7246 = internal global [2 x i8] c"\0A\00"

define void @dummy([0 x i8]* nocapture readonly %vals_7224) local_unnamed_addr {
dummy_start:
  %0 = getelementptr inbounds [0 x i8], [0 x i8]* %vals_7224, i64 0, i64 6
  %1 = load i8, i8* %0, align 1
  tail call void @anydsl_print_char(i8 %1)
  tail call void @anydsl_print_string([0 x i8]* bitcast ([2 x i8]* @_7246 to [0 x i8]*))
  ret void
}

declare void @anydsl_print_char(i8) local_unnamed_addr

declare void @anydsl_print_string([0 x i8]*) local_unnamed_addr

```





**Cuda variant:**

**Range:**
.cu code :

```

extern "C" {
typedef struct {
    int e[1];
} array_5547;
typedef struct {
    unsigned char e[9];
} array_5543;

__device__ inline int threadIdx_x() { return threadIdx.x; }
__device__ inline int threadIdx_y() { return threadIdx.y; }
__device__ inline int threadIdx_z() { return threadIdx.z; }
__device__ inline int blockIdx_x() { return blockIdx.x; }
__device__ inline int blockIdx_y() { return blockIdx.y; }
__device__ inline int blockIdx_z() { return blockIdx.z; }
__device__ inline int blockDim_x() { return blockDim.x; }
__device__ inline int blockDim_y() { return blockDim.y; }
__device__ inline int blockDim_z() { return blockDim.z; }
__device__ inline int gridDim_x() { return gridDim.x; }
__device__ inline int gridDim_y() { return gridDim.y; }
__device__ inline int gridDim_z() { return gridDim.z; }
__global__ void lambda_9908(char*);

__global__ __launch_bounds__ (1 * 1 * 1) void lambda_9908(char* _9911_10879) {
    array_5547* buf__10914;
    buf__10914 = (array_5547*)_9911_10879;
    
    int* _10916;
    _10916 = &buf__10914->e[0];
    int b_10885_slot;
    int* b_10885;
    b_10885 = &b_10885_slot;
    *b_10885 = 0;
    goto l10880;
    l10880: ;
        int _10886;
        _10886 = *b_10885;
        int _10887;
        _10887 = _10886;
        bool _10889;
        _10889 = _10887 < 9;
        if (_10889) goto l10890; else goto l10919;
    l10919: ;
        return ;
    l10890: ;
        int _10903;
        _10903 = *b_10885;
        int _10904;
        _10904 = _10903;
        array_5543 _10900_3;
        {
        array_5543 _10900_3_tmp = { { 49, 50, 51, 52, 53, 54, 55, 56, 0, } };
         _10900_3 = _10900_3_tmp;
        }
        
        unsigned char _10905;
        _10905 = _10900_3.e[_10904];
        int _10912;
        _10912 = 1 + _10904;
        bool _10906;
        _10906 = _10905 == 55;
        *b_10885 = _10912;
        if (_10906) goto l10907; else goto l10918;
    l10918: ;
        goto l10908;
    l10907: ;
        *_10916 = _10904;
        goto l10908;
    l10908: ;
        goto l10880;
}

}

```

**Unroll:**
```
extern "C" {
typedef struct {
    int e[1];
} array_4877;

__device__ inline int threadIdx_x() { return threadIdx.x; }
__device__ inline int threadIdx_y() { return threadIdx.y; }
__device__ inline int threadIdx_z() { return threadIdx.z; }
__device__ inline int blockIdx_x() { return blockIdx.x; }
__device__ inline int blockIdx_y() { return blockIdx.y; }
__device__ inline int blockIdx_z() { return blockIdx.z; }
__device__ inline int blockDim_x() { return blockDim.x; }
__device__ inline int blockDim_y() { return blockDim.y; }
__device__ inline int blockDim_z() { return blockDim.z; }
__device__ inline int gridDim_x() { return gridDim.x; }
__device__ inline int gridDim_y() { return gridDim.y; }
__device__ inline int gridDim_z() { return gridDim.z; }
__global__ void lambda_9088(char*);

__global__ __launch_bounds__ (1 * 1 * 1) void lambda_9088(char* _9091_9400) {
    array_4877* buf__9401;
    buf__9401 = (array_4877*)_9091_9400;
    int* _9403;
    _9403 = &buf__9401->e[0];
    *_9403 = 6;
    return ;
}

}

```


